
<a href="https://colab.research.google.com/github/takzen/financial-ai-engineering-showcase/blob/main/notebooks/week_06_agents/06_project_autonomous_analyst.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🏆 Tydzień 6, Dzień 6: Projekt - Autonomiczny Analityk (Web -> PDF)

To finałowy projekt modułu Agents. Zbudujemy system, który wykonuje pracę od A do Z.

**Architektura:**
1.  **Researcher Agent:** Szuka informacji w sieci (Tavily).
2.  **Writer Agent:** Analizuje dane i pisze treść raportu.
3.  **Publisher Tool:** Specjalne narzędzie (funkcja Python), które Writer może wywołać, aby zapisać tekst do pliku PDF.

**Efekt końcowy:** Plik PDF na dysku z najświeższymi danymi giełdowymi, stworzony bez udziału człowieka.

---
### 🛠️ 1. Instalacja

Potrzebujemy `reportlab` do generowania PDFów.

In [ ]:
!uv add langgraph langchain langchain-google-genai langchain-community tavily-python reportlab python-dotenv

### ⚙️ 2. Konfiguracja

Ładujemy klucze i model. Dodajemy też naszą funkcję czyszczącą tekst, żeby logi były czytelne.

In [1]:
import os
import warnings
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
# Stabilny import Tavily
from langchain_community.tools.tavily_search import TavilySearchResults
import time

warnings.filterwarnings("ignore")

# Ścieżki
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "../../"))
load_dotenv(os.path.join(project_root, ".env"))

# Sprawdzenie kluczy
if not os.getenv("GOOGLE_API_KEY") or not os.getenv("TAVILY_API_KEY"):
    print("❌ BŁĄD: Brak kluczy w .env")
else:
    print("✅ Klucze załadowane.")

# Model Gemini 2.5 Flash
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# Narzędzie Web Search
web_search = TavilySearchResults(k=5)

# Helper do czyszczenia odpowiedzi (JSON -> Text)
def parse_gemini_content(content):
    if isinstance(content, str): return content
    elif isinstance(content, list): return "".join([b.get("text", "") for b in content if "text" in b])
    return str(content)

✅ Klucze załadowane.


### 📄 3. Custom Tool: PDF Generator

To jest kluczowy moment. Dajemy Agentowi "ręce".
Stworzymy funkcję `save_to_pdf`, udekorujemy ją jako `@tool` i damy Writerowi.

In [2]:
from langchain_core.tools import tool
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
import textwrap

@tool
def save_to_pdf(filename: str, content: str) -> str:
    """
    Zapisuje podany tekst do pliku PDF.
    Użyj tego narzędzia, gdy masz gotowy raport i chcesz go opublikować.
    filename: nazwa pliku (np. 'raport_apple.pdf').
    content: treść raportu.
    """
    try:
        # Ustalamy ścieżkę zapisu (folder data/reports)
        reports_dir = os.path.join(project_root, "data", "reports")
        os.makedirs(reports_dir, exist_ok=True)
        file_path = os.path.join(reports_dir, filename)
        
        c = canvas.Canvas(file_path, pagesize=A4)
        width, height = A4
        y = height - 50
        
        # Prosty nagłówek
        c.setFont("Helvetica-Bold", 16)
        c.drawString(50, y, f"RAPORT: {filename}")
        y -= 40
        
        # Treść (proste łamanie linii)
        c.setFont("Helvetica", 11)
        
        # Dzielimy tekst na akapity i linie (textwrap)
        lines = content.split('\n')
        for line in lines:
            # Zawijanie długich linii do 90 znaków
            wrapped_lines = textwrap.wrap(line, width=90)
            for w_line in wrapped_lines:
                if y < 50: # Nowa strona
                    c.showPage()
                    c.setFont("Helvetica", 11)
                    y = height - 50
                
                c.drawString(50, y, w_line)
                y -= 15
            y -= 5 # Odstęp po akapicie
            
        c.save()
        return f"✅ Sukces! Raport zapisano w: {file_path}"
    except Exception as e:
        return f"❌ Błąd zapisu PDF: {str(e)}"

print("✅ Narzędzie PDF gotowe.")

✅ Narzędzie PDF gotowe.


### 🕵️‍♂️ 4. Definicja Agentów (Nodes)

1.  **Researcher:** Używa `web_search`. Zbiera dane.
2.  **Writer:** Używa `save_to_pdf`. Pisze raport i go zapisuje.

In [3]:
from langchain_core.messages import SystemMessage

# --- RESEARCHER ---
# Ten model ma dostęp TYLKO do wyszukiwarki
llm_researcher = llm.bind_tools([web_search])

def researcher_node(state):
    time.sleep(3) # Czekaj 5 sekund przed każdym ruchem
    messages = state["messages"]
    prompt = [
        SystemMessage(content="""
        Jesteś Researcherem Rynkowym.
        Twoim zadaniem jest znalezienie najnowszych informacji finansowych o firmie wskazanej przez użytkownika.
        Szukaj: Ceny akcji, ostatnich wyników kwartalnych, głównych newsów z ostatniego tygodnia.
        Zwracaj same fakty.
        """)
    ] + messages
    return {"messages": [llm_researcher.invoke(prompt)]}

print("✅ Researcher zdefiniowany.")

✅ Researcher zdefiniowany.


In [4]:
# --- WRITER ---
# Ten model ma dostęp TYLKO do narzędzia PDF
llm_writer = llm.bind_tools([save_to_pdf])

def writer_node(state):
    time.sleep(3) # Czekaj 5 sekund przed każdym ruchem
    messages = state["messages"]
    prompt = [
        SystemMessage(content="""
        Jesteś Analitykiem i Wydawcą.
        Na podstawie informacji od Researchera, napisz profesjonalny raport w języku POLSKIM.
        
        Po napisaniu treści, MUSISZ użyć narzędzia 'save_to_pdf', aby zapisać raport na dysku.
        Nazwij plik sensownie, np. 'Raport_NazwaFirmy.pdf'.
        W treści PDF nie używaj Markdown (pogrubień **), używaj czystego tekstu.
        """)
    ] + messages
    return {"messages": [llm_writer.invoke(prompt)]}

print("✅ Writer zdefiniowany.")

✅ Writer zdefiniowany.


### 🔗 5. Budowa Grafu

Przepływ jest nieco bardziej skomplikowany, bo mamy dwa różne zestawy narzędzi.
`Researcher` -> (decyzja) -> `Web Tools`
`Writer` -> (decyzja) -> `PDF Tool`

In [5]:
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

class AgentState(TypedDict):
    messages: Annotated[List, add_messages]

builder = StateGraph(AgentState)

# Węzły
builder.add_node("researcher", researcher_node)
builder.add_node("web_tools", ToolNode([web_search])) # Węzeł wykonujący szukanie

builder.add_node("writer", writer_node)
builder.add_node("pdf_tool", ToolNode([save_to_pdf])) # Węzeł wykonujący zapis PDF

# Krawędzie

# 1. Start -> Researcher
builder.add_edge(START, "researcher")

# 2. Researcher -> (Szukaj dalej LUB idź do Writera)
def route_researcher(state):
    last_msg = state["messages"][-1]
    # Jeśli chce użyć narzędzia -> idź do web_tools
    if last_msg.tool_calls:
        return "web_tools"
    # Jeśli skończył -> idź do writera
    return "writer"

builder.add_conditional_edges("researcher", route_researcher, ["web_tools", "writer"])
builder.add_edge("web_tools", "researcher") # Pętla zwrotna

# 3. Writer -> (Zapisz PDF LUB Koniec)
def route_writer(state):
    last_msg = state["messages"][-1]
    if last_msg.tool_calls:
        return "pdf_tool"
    return END

builder.add_conditional_edges("writer", route_writer, ["pdf_tool", END])
builder.add_edge("pdf_tool", "writer") # Pętla zwrotna (żeby potwierdzić zapis)

# Kompilacja
app = builder.compile()
print("✅ Graf gotowy.")

✅ Graf gotowy.


### 🚀 6. Wielki Finał: Uruchomienie

Poprosimy Agenta o raport o firmie **Microsoft (MSFT)**.
Spodziewamy się, że:
1.  Znajdzie aktualne dane.
2.  Stworzy plik PDF w `data/reports/`.

In [ ]:
from langchain_core.messages import HumanMessage

query = "Przygotuj raport o obecnej sytuacji Microsoftu (MSFT). Skup się na inwestycjach w AI."

print(f"🚀 START: {query}\n")

events = app.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="values"
)

for event in events:
    last_msg = event["messages"][-1]
    
    if last_msg.type == "ai":
        if last_msg.tool_calls:
            tool_name = last_msg.tool_calls[0]["name"]
            print(f"🤖 AI: Używam narzędzia -> {tool_name}")
        else:
            clean_text = parse_gemini_content(last_msg.content)
            if clean_text.strip():
                print(f"\n💬 AI MÓWI:\n{clean_text}")
            
    elif last_msg.type == "tool":
        # Pokazujemy tylko początek wyniku narzędzia
        print(f"🛠️ WYNIK NARZĘDZIA: {str(last_msg.content)[:100]}...")

### ✅ Weryfikacja

Sprawdźmy, czy plik PDF naprawdę powstał w systemie plików.

In [7]:
report_path = os.path.join(project_root, "data", "reports")
files = os.listdir(report_path)

print(f"📂 Pliki w {report_path}:")
for f in files:
    if f.endswith(".pdf"):
        print(f"   📄 {f}")

📂 Pliki w c:\Users\takze\OneDrive\Pulpit\project\financial-ai-engineering\data\reports:
   📄 balance_sheet.pdf
   📄 financial_report_q3.pdf
   📄 Risk_Dashboard.pdf
   📄 Risk_Dashboard_OPTIMIZED.pdf
   📄 tech_corp_10k.pdf


## 🧠 Zadanie Domowe: Krytyk Raportów

Agent czasami generuje zbyt proste raporty.
Dodaj węzeł **Reviewer**, który wchodzi do gry **przed** zapisaniem PDF.

**Zadanie:**
1. Zmień graf tak, aby po `writer` (gdy wygeneruje tekst, ale przed tool call) następował `reviewer`.
2. Reviewer ma sprawdzić, czy tekst zawiera sekcję "Ryzyka". Jeśli nie – cofa do Writera z instrukcją "Dodaj sekcję o ryzykach".
3. To zadanie wymaga zaawansowanej edycji grafu (dla chętnych).

In [ ]:
# --- ROZWIĄZANIE ZADANIA DOMOWEGO: Reviewer Gatekeeper ---

# 1. Definicja Węzła Reviewera
def reviewer_node(state):
    messages = state["messages"]
    last_message = messages[-1]
    
    # Prompt sprawdzający jakość
    prompt = [
        SystemMessage(content="""
        Jesteś surowym Edytorem (Reviewer).
        Twój cel: Sprawdzić, czy raport napisany przez Analityka (Writer) zawiera sekcję o RYZYKACH.
        
        Zasady:
        1. Przeczytaj ostatnią wiadomość.
        2. Jeśli tekst zawiera sekcję "Ryzyka" (lub "Risks", "Zagrożenia") -> Odpowiedz tylko: "APPROVE".
        3. Jeśli NIE ma tej sekcji -> Odpowiedz: "BŁĄD: Brakuje sekcji o ryzykach inwestycyjnych. Dodaj ją natychmiast."
        """)
    ] + messages
    
    response = llm.invoke(prompt)
    return {"messages": [response]}

# 2. Definicja Węzła Publishera (Zapis PDF)
# Ten węzeł uruchomi się TYLKO, gdy Reviewer da zgodę
def publisher_node(state):
    messages = state["messages"]
    
    # Szukamy ostatniego raportu (tekstu Writera) w historii
    # Cofamy się, aż znajdziemy tekst Writera (omijając wiadomości Reviewera)
    report_content = "Brak treści"
    for msg in reversed(messages):
        if isinstance(msg, AIMessage) and "BŁĄD" not in str(msg.content) and "APPROVE" not in str(msg.content):
            report_content = str(msg.content)
            break
            
    print("🖨️ PUBLISHER: Zapisuję zatwierdzony raport...")
    
    # Wywołujemy narzędzie bezpośrednio
    filename = "Zatwierdzony_Raport_Msft.pdf"
    result = save_to_pdf.invoke({"filename": filename, "content": report_content})
    
    return {"messages": [AIMessage(content=f"Raport opublikowany: {result}")]}


# 3. Logika Przejścia (Router)
def route_reviewer(state):
    last_msg = state["messages"][-1]
    content = str(last_msg.content)
    
    if "APPROVE" in content:
        return "publisher"
    return "writer" # Cofnij do poprawki

# --- BUDOWA NOWEGO GRAFU ---
builder_v2 = StateGraph(AgentState)

# Węzły
builder_v2.add_node("researcher", researcher_node)
builder_v2.add_node("web_tools", ToolNode([web_search]))
builder_v2.add_node("writer", writer_node)
builder_v2.add_node("reviewer", reviewer_node)  # <-- NOWOŚĆ
builder_v2.add_node("publisher", publisher_node) # <-- NOWOŚĆ

# Krawędzie
builder_v2.add_edge(START, "researcher")

# Researcher -> WebTools lub Writer
builder_v2.add_conditional_edges(
    "researcher",
    route_researcher,
    {"web_tools": "web_tools", "writer": "writer"}
)
builder_v2.add_edge("web_tools", "researcher")

# Writer -> Reviewer (Zamiast od razu do PDF)
builder_v2.add_edge("writer", "reviewer")

# Reviewer -> Publisher (jeśli OK) lub Writer (jeśli poprawka)
builder_v2.add_conditional_edges(
    "reviewer",
    route_reviewer,
    {"publisher": "publisher", "writer": "writer"}
)

builder_v2.add_edge("publisher", END)

# Kompilacja
app_v2 = builder_v2.compile()
print("✅ Graf z Reviewerem (Pętla Korekcyjna) gotowy.")

# --- TEST ---
from langchain_core.messages import AIMessage # Import potrzebny do logiki publishera

query = "Napisz raport o Microsoft. (Na początku celowo nie pisz o ryzykach, żeby sprawdzić czy Reviewer to wyłapie!)"
print(f"\n🚀 TEST REVIEWERA: {query}\n")

events = app_v2.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="values"
)

for event in events:
    last_msg = event["messages"][-1]
    if last_msg.type == "ai":
        content = str(last_msg.content)
        if "APPROVE" in content:
            print(f"👮‍♂️ REVIEWER: {content} (Idziemy do druku!)")
        elif "BŁĄD" in content:
            print(f"👮‍♂️ REVIEWER: {content} (Cofam do Writera!)")
        elif "Raport opublikowany" in content:
            print(f"✅ FINAL: {content}")